In [1]:
import pandas as pd
import xgboost as xgb

# Cargar el dataset
df = pd.read_csv('../data/ventas_featurev2.csv', sep=',')

In [2]:
# Convertir 'periodo' de fecha a entero tipo YYYYMM
df["periodo"] = pd.to_datetime(df["periodo"])
df["periodo"] = df["periodo"].dt.strftime("%Y%m").astype(int)


In [3]:
df.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,cust_request_qty,cust_request_tn,cat1,cat2,cat3,...,delta_tn_lag_33,delta_tn_lag_34,delta_tn_lag_35,ipc,var_mensual,var_anual,promedio_mens_dolar_venta,temp_media,temp_max_media,temp_min_media
0,201701,10001,20001,0.0,99.43861,11.0,99.43861,HC,ROPA LAVADO,Liquido,...,NaN,NaN,NaN,101.5859,1.59,24.8,15.91,25.65,30.4,20.9
1,201701,10002,20001,0.0,35.72806,17.0,38.68301,HC,ROPA LAVADO,Liquido,...,NaN,NaN,NaN,101.5859,1.59,24.8,15.91,25.65,30.4,20.9
2,201701,10003,20001,0.0,143.49426,17.0,143.49426,HC,ROPA LAVADO,Liquido,...,NaN,NaN,NaN,101.5859,1.59,24.8,15.91,25.65,30.4,20.9
3,201701,10004,20001,0.0,184.72927,9.0,184.72927,HC,ROPA LAVADO,Liquido,...,NaN,NaN,NaN,101.5859,1.59,24.8,15.91,25.65,30.4,20.9
4,201701,10005,20001,0.0,19.08407,23.0,19.08407,HC,ROPA LAVADO,Liquido,...,NaN,NaN,NaN,101.5859,1.59,24.8,15.91,25.65,30.4,20.9


In [3]:
# 1. Crear dataset de entrenamiento
df_train = df.copy()
df_train2 = df.copy()

In [5]:
df_train.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,cust_request_qty,cust_request_tn,cat1,cat2,cat3,...,delta_tn_lag_33,delta_tn_lag_34,delta_tn_lag_35,ipc,var_mensual,var_anual,promedio_mens_dolar_venta,temp_media,temp_max_media,temp_min_media
0,201701,10001,20001,0.0,99.43861,11.0,99.43861,HC,ROPA LAVADO,Liquido,...,NaN,NaN,NaN,101.5859,1.59,24.8,15.91,25.65,30.4,20.9
1,201701,10002,20001,0.0,35.72806,17.0,38.68301,HC,ROPA LAVADO,Liquido,...,NaN,NaN,NaN,101.5859,1.59,24.8,15.91,25.65,30.4,20.9
2,201701,10003,20001,0.0,143.49426,17.0,143.49426,HC,ROPA LAVADO,Liquido,...,NaN,NaN,NaN,101.5859,1.59,24.8,15.91,25.65,30.4,20.9
3,201701,10004,20001,0.0,184.72927,9.0,184.72927,HC,ROPA LAVADO,Liquido,...,NaN,NaN,NaN,101.5859,1.59,24.8,15.91,25.65,30.4,20.9
4,201701,10005,20001,0.0,19.08407,23.0,19.08407,HC,ROPA LAVADO,Liquido,...,NaN,NaN,NaN,101.5859,1.59,24.8,15.91,25.65,30.4,20.9


In [4]:
df["periodo"] = df["periodo"].astype(int)

In [5]:
# Convierte todas las columnas object a category
for col in df.select_dtypes(include='object').columns:
    df[col] = df[col].astype('category')


In [6]:
X_train = df_train2.drop(columns=["tn", "periodo", "descripcion", "prod_cust"], errors="ignore")
y_train = df_train["tn"]

# 2. Generar dataset de predicción para 202002
# Tomamos los registros de 201912 como base y simulamos las features
df_201912 = df[df["periodo"] == 201912].copy()
df_pred = df_201912.copy()
df_pred["periodo"] = 202002  # Simulamos el período futuro

In [9]:
df_pred.head()

,periodo,customer_id,product_id,plan_precios_cuidados,tn,cust_request_qty,cust_request_tn,cat1,cat2,cat3,...,delta_tn_lag_33,delta_tn_lag_34,delta_tn_lag_35,ipc,var_mensual,var_anual,promedio_mens_dolar_venta,temp_media,temp_max_media,temp_min_media
20895,202002,10001,20001,0.0,180.21938,18.0,214.72185,HC,ROPA LAVADO,Liquido,...,180.21938,180.21938,180.21938,283.4442,3.74,53.83,59.88,24.0,29.5,18.0
20896,202002,10002,20001,0.0,113.33165,20.0,115.30322,HC,ROPA LAVADO,Liquido,...,113.33165,113.33165,113.33165,283.4442,3.74,53.83,59.88,24.0,29.5,18.0
20897,202002,10003,20001,0.0,102.27517,9.0,113.98137,HC,ROPA LAVADO,Liquido,...,102.15195,102.27517,102.27517,283.4442,3.74,53.83,59.88,24.0,29.5,18.0
20898,202002,10004,20001,0.0,34.64810,8.0,34.64810,HC,ROPA LAVADO,Liquido,...,34.64810,34.52488,34.64810,283.4442,3.74,53.83,59.88,24.0,29.5,18.0
20899,202002,10005,20001,0.0,19.60368,25.0,19.60368,HC,ROPA LAVADO,Liquido,...,19.60368,19.60368,19.48046,283.4442,3.74,53.83,59.88,24.0,29.5,18.0


In [7]:
cat_cols = ["cat1", "cat2", "cat3", "brand"]
for col in cat_cols:
    if col in X_train.columns:
        X_train[col] = X_train[col].astype("category")

In [11]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 16763760 entries, 0 to 16763759
Data columns (total 93 columns):
 #   Column                     Dtype   
---  ------                     -----   
 0   customer_id                int64   
 1   product_id                 int64   
 2   plan_precios_cuidados      float64 
 3   cust_request_qty           float64 
 4   cust_request_tn            float64 
 5   cat1                       category
 6   cat2                       category
 7   cat3                       category
 8   brand                      category
 9   sku_size                   int64   
 10  productos_estrella         int64   
 11  mes                        int64   
 12  catastrofe                 int64   
 13  cliente_estrella           int64   
 14  tn_lag_1                   float64 
 15  tn_lag_2                   float64 
 16  tn_lag_3                   float64 
 17  tn_lag_4                   float64 
 18  tn_lag_5                   float64 
 19  tn_lag_6           

In [12]:
from sklearn.model_selection import GridSearchCV
from xgboost import XGBRegressor

# Subsample del dataset para que no explote (¡no uses los 16M en GridSearch!)
X_sample = X_train.sample(n=100_000, random_state=42)
y_sample = y_train.loc[X_sample.index]

In [13]:


param_grid = {
    "max_depth": [8, 10, 12, 14],
    "learning_rate": [0.01, 0.1, 0.2],
    "n_estimators": [150, 200, 250],
    "subsample": [0.6, 0.8, 1.0],
}

model = XGBRegressor(enable_categorical=True, n_jobs=-1)

grid_search = GridSearchCV(
    estimator=model,
    param_grid=param_grid,
    scoring="neg_root_mean_squared_error",
    cv=3,
    verbose=1
)

grid_search.fit(X_sample, y_sample)

print("✅ Mejor combinación:", grid_search.best_params_)
print("📉 Mejor score (RMSE negativo):", grid_search.best_score_)


Fitting 3 folds for each of 108 candidates, totalling 324 fits
✅ Mejor combinación: {'learning_rate': 0.2, 'max_depth': 12, 'n_estimators': 150, 'subsample': 0.6}
📉 Mejor score (RMSE negativo): -0.4888595725648424


In [14]:
# Eliminar la columna de respuesta original (tn)
df_pred = df_pred.drop(columns=["tn"], errors="ignore")

# 3. Crear X_pred antes de convertir tipos
X_pred = df_pred.drop(columns=["periodo", "descripcion", "prod_cust"], errors="ignore")

# 4. Convertir columnas categóricas a dtype 'category'
cat_cols = ["cat1", "cat2", "cat3", "brand"]
for col in cat_cols:
    if col in X_train.columns:
        X_train[col] = X_train[col].astype("category")
        X_pred[col] = X_pred[col].astype("category")
        
# 3. Entrenar el modelo
model = xgb.XGBRegressor(n_estimators=150, learning_rate=0.2, max_depth=12, n_jobs=-1, subsample=0.6, random_state=42, enable_categorical=True)
model.fit(X_train, y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=None,
             enable_categorical=True, eval_metric=None, feature_types=None,
             feature_weights=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=0.2, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=12,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=150,
             n_jobs=-1, num_parallel_tree=None, ...)

In [15]:
print("✅ X_train shape:", X_train.shape)
print("✅ X_pred shape:", X_pred.shape)
print("🔍 Columns in X_train:", X_train.columns.tolist())
print("🔍 Columns in X_pred:", X_pred.columns.tolist())
print("🔍 Missing columns in X_pred:", set(X_train.columns) - set(X_pred.columns))

✅ X_train shape: (16763760, 93)
✅ X_pred shape: (465660, 93)
🔍 Columns in X_train: ['customer_id', 'product_id', 'plan_precios_cuidados', 'cust_request_qty', 'cust_request_tn', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size', 'productos_estrella', 'mes', 'catastrofe', 'cliente_estrella', 'tn_lag_1', 'tn_lag_2', 'tn_lag_3', 'tn_lag_4', 'tn_lag_5', 'tn_lag_6', 'tn_lag_7', 'tn_lag_8', 'tn_lag_9', 'tn_lag_10', 'tn_lag_11', 'tn_lag_12', 'tn_lag_13', 'tn_lag_14', 'tn_lag_15', 'tn_lag_16', 'tn_lag_17', 'tn_lag_18', 'tn_lag_19', 'tn_lag_20', 'tn_lag_21', 'tn_lag_22', 'tn_lag_23', 'tn_lag_24', 'tn_lag_25', 'tn_lag_26', 'tn_lag_27', 'tn_lag_28', 'tn_lag_29', 'tn_lag_30', 'tn_lag_31', 'tn_lag_32', 'tn_lag_33', 'tn_lag_34', 'tn_lag_35', 'tn_lag_36', 'tn_mas_2', 'delta_tn_lag_1', 'delta_tn_lag_2', 'delta_tn_lag_3', 'delta_tn_lag_4', 'delta_tn_lag_5', 'delta_tn_lag_6', 'delta_tn_lag_7', 'delta_tn_lag_8', 'delta_tn_lag_9', 'delta_tn_lag_10', 'delta_tn_lag_11', 'delta_tn_lag_12', 'delta_tn_lag_13', 'delta

In [17]:
# 4. Hacer predicciones sobre 202002

df_pred["tn_pred"] = model.predict(X_pred)

# 5. Agrupar por product_id y exportar resultados
df_result = (
    df_pred.groupby("product_id")["tn_pred"]
    .sum()
    .reset_index()
    .rename(columns={"tn_pred": "tn"})
)

df_result.to_csv("../data/submission_xgboost_202002_lb_v4.csv", index=False)

print("✅ Archivo generado: submission_xgboost_202002_lb_v4.csv")


✅ Archivo generado: submission_xgboost_202002_lb_v4.csv


In [8]:
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMRegressor

# Renombrar para evitar conflictos
Xtrain_lgb = X_train.copy()
ytrain_lgb = y_train.copy()

# Definir el modelo base
lgb_base_model = LGBMRegressor(objective='regression', random_state=42)

# Espacio ampliado de hiperparámetros
param_grid_lgb = {
    'learning_rate': [0.05, 0.1, 0.15, 0.2],
    'max_depth': [6, 8, 10, 12],
    'n_estimators': [100, 150, 200],
    'subsample': [0.6, 0.7, 0.8, 0.9],
    'colsample_bytree': [0.6, 0.8, 1.0]
}

# Búsqueda con GridSearchCV
grid_search_lgb = GridSearchCV(
    estimator=lgb_base_model,
    param_grid=param_grid_lgb,
    scoring='neg_root_mean_squared_error',  # mejor para regresión
    cv=3,
    verbose=1,
    n_jobs=-1
)

# Ejecutar búsqueda
print("🔍 Ejecutando GridSearchCV ampliado para LightGBM...")
grid_search_lgb.fit(Xtrain_lgb, ytrain_lgb)
print("✅ GridSearch finalizado.")
print(f"🏆 Mejor combinación: {grid_search_lgb.best_params_}")
print(f"📉 Mejor score (RMSE negativo): {grid_search_lgb.best_score_}")


/opt/anaconda3/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(


🔍 Ejecutando GridSearchCV ampliado para LightGBM...
Fitting 3 folds for each of 576 candidates, totalling 1728 fits


/opt/anaconda3/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your current version.
  warnings.warn(
/opt/anaconda3/lib/python3.11/site-packages/dask/dataframe/_pyarrow_compat.py:23: UserWarning: You are using pyarrow version 11.0.0 which is known to be insecure. See https://www.cve.org/CVERecord?id=CVE-2023-47248 for further details. Please upgrade to pyarrow>=14.0.1 or install pyarrow-hotfix to patch your cu

: 